<a href="https://colab.research.google.com/github/soohyunme/TensorFlow_Tutorial/blob/main/Code/10_Transfer_Learning_Fine_Tuning_TF_Hub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
import tensorflow_hub as hub

# Device setting

In [19]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0],True)

#Model Training

## Load data

In [20]:
(x_train,y_train), (x_test,y_test) = mnist.load_data()

## Normalize

In [21]:
x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0

## Model

In [22]:
def my_model():
  inputs = keras.Input(shape=(28, 28, 1))
  x = layers.Conv2D(32, 3, padding='same')(inputs)
  x = layers.Conv2D(64, 3, padding='same')(x)
  x = layers.MaxPooling2D()(x)

  x = layers.Conv2D(128, 3, padding='same')(x)
  x = layers.Conv2D(256, 3, padding='same')(x)
  x = layers.MaxPooling2D()(x)

  x = layers.Flatten()(x)
  outputs = layers.Dense(10)(x)
  model = keras.Model(inputs=inputs, outputs=outputs)
  
  return model

model = my_model()

model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer = keras.optimizers.Adam(),
    metrics = ['accuracy'],
)

model.fit(x_train, y_train, batch_size=32, epochs=2, verbose=2)
model.evaluate(x_test, y_test, batch_size=32, verbose=2)
model.summary()


Epoch 1/2
1875/1875 - 24s - loss: 0.1366 - accuracy: 0.9592 - 24s/epoch - 13ms/step
Epoch 2/2
1875/1875 - 23s - loss: 0.0800 - accuracy: 0.9756 - 23s/epoch - 12ms/step
313/313 - 2s - loss: 0.0639 - accuracy: 0.9798 - 2s/epoch - 6ms/step
Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_846 (Conv2D)         (None, 28, 28, 32)        320       
                                                                 
 conv2d_847 (Conv2D)         (None, 28, 28, 64)        18496     
                                                                 
 max_pooling2d_36 (MaxPoolin  (None, 14, 14, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_848 (Conv2D)

## Model Save

In [23]:
model.save('pretrained/')

INFO:tensorflow:Assets written to: pretrained/assets


# Pretrained Model

## Load data

In [24]:
(x_train,y_train), (x_test,y_test) = mnist.load_data()

## Normalize

In [25]:
x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0

## Load Model

In [26]:
model = keras.models.load_model('pretrained/')

Layer Freeze

In [27]:
model.trainable = False

# 반복문을 활용한 방법 - 예제의 경우 모든 Layer가 trainable == True
for layer in model.layers:
  assert layer.trainable == False
  layer.trainable = False

In [28]:
print(model.summary())

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_846 (Conv2D)         (None, 28, 28, 32)        320       
                                                                 
 conv2d_847 (Conv2D)         (None, 28, 28, 64)        18496     
                                                                 
 max_pooling2d_36 (MaxPoolin  (None, 14, 14, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_848 (Conv2D)         (None, 14, 14, 128)       73856     
                                                                 
 conv2d_849 (Conv2D)         (None, 14, 14, 256)       295168    
                                                           

In [29]:
model.trainable = False # Layer를 Freeze하면 실행 시간이 빨라짐

base_inputs = model.layers[0].input
base_outputs = model.layers[-2].output
final_outputs = layers.Dense(10)(base_outputs)

new_model = keras.Model(inputs=base_inputs, outputs=final_outputs)

new_model.compile(
    optimizer = keras.optimizers.Adam(),
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)

new_model.fit(x_train, y_train, batch_size=32, epochs=3, verbose=2)

Epoch 1/3
1875/1875 - 9s - loss: 0.0706 - accuracy: 0.9796 - 9s/epoch - 5ms/step
Epoch 2/3
1875/1875 - 9s - loss: 0.0304 - accuracy: 0.9905 - 9s/epoch - 5ms/step
Epoch 3/3
1875/1875 - 9s - loss: 0.0216 - accuracy: 0.9930 - 9s/epoch - 5ms/step


In [30]:
model.trainable = True

base_inputs = model.layers[0].input
base_outputs = model.layers[-2].output
final_outputs = layers.Dense(10)(base_outputs)

new_model = keras.Model(inputs=base_inputs, outputs=final_outputs)

new_model.compile(
    optimizer = keras.optimizers.Adam(),
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)

new_model.fit(x_train, y_train, batch_size=32, epochs=3, verbose=2)

Epoch 1/3
1875/1875 - 23s - loss: 0.0950 - accuracy: 0.9714 - 23s/epoch - 12ms/step
Epoch 2/3
1875/1875 - 23s - loss: 0.0706 - accuracy: 0.9795 - 23s/epoch - 12ms/step
Epoch 3/3
1875/1875 - 23s - loss: 0.0655 - accuracy: 0.9823 - 23s/epoch - 12ms/step


# Pretrained Keras Model

In [43]:
x = tf.random.normal(shape=(5, 299, 299, 3))
y = tf.constant([0, 1, 2, 3, 4])

model = keras.applications.InceptionV3(include_top=True) # include_top -> 가장 상단의 fully connected계층들을 포함 시킬지의 여부
incept_base_inputs = model.layers[0].input
incept_base_outputs = model.layers[-2].output
incept_final_outputs = layers.Dense(5)(incept_base_outputs)
incept_model = keras.Model(inputs=incept_base_inputs, outputs=incept_base_outputs)

incept_model.compile(
    optimizer = keras.optimizers.Adam(),
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)
incept_model.fit(x, y, batch_size=1, epochs=15, verbose=2)



Epoch 1/15
5/5 - 9s - loss: 7.6250 - accuracy: 0.0000e+00 - 9s/epoch - 2s/step
Epoch 2/15
5/5 - 1s - loss: 7.5175 - accuracy: 0.0000e+00 - 517ms/epoch - 103ms/step
Epoch 3/15
5/5 - 1s - loss: 7.4339 - accuracy: 0.0000e+00 - 514ms/epoch - 103ms/step
Epoch 4/15
5/5 - 1s - loss: 7.3281 - accuracy: 0.2000 - 512ms/epoch - 102ms/step
Epoch 5/15
5/5 - 1s - loss: 7.2810 - accuracy: 0.6000 - 510ms/epoch - 102ms/step
Epoch 6/15
5/5 - 1s - loss: 7.2550 - accuracy: 0.8000 - 510ms/epoch - 102ms/step
Epoch 7/15
5/5 - 1s - loss: 7.2420 - accuracy: 1.0000 - 511ms/epoch - 102ms/step
Epoch 8/15
5/5 - 1s - loss: 7.2321 - accuracy: 1.0000 - 509ms/epoch - 102ms/step
Epoch 9/15
5/5 - 1s - loss: 7.2262 - accuracy: 1.0000 - 509ms/epoch - 102ms/step
Epoch 10/15
5/5 - 1s - loss: 7.2197 - accuracy: 1.0000 - 510ms/epoch - 102ms/step
Epoch 11/15
5/5 - 1s - loss: 7.2150 - accuracy: 1.0000 - 512ms/epoch - 102ms/step
Epoch 12/15
5/5 - 1s - loss: 7.2122 - accuracy: 1.0000 - 517ms/epoch - 103ms/step
Epoch 13/15
5/5 - 1

# Pretrained Hub Model

In [45]:
x = tf.random.normal(shape=(5, 299, 299, 3))
y = tf.constant([0, 1, 2, 3, 4])
url = 'https://tfhub.dev/google/inaturalist/inception_v3/feature_vector/5'

In [53]:
base_model = hub.KerasLayer(url,input_shape=(299, 299, 3))
base_model.trainable = False

model = keras.Sequential([
      base_model,
      layers.Dense(128, activation='relu'),
      layers.Dense(64, activation='relu'),
      layers.Dense(5),
])

In [54]:
model.compile(
    optimizer = keras.optimizers.Adam(),
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics = ['accuracy']
)

In [55]:
model.fit(x, y, batch_size=1, epochs=15, verbose=2)

Epoch 1/15
5/5 - 6s - loss: 2.2136 - accuracy: 0.0000e+00 - 6s/epoch - 1s/step
Epoch 2/15
5/5 - 0s - loss: 1.7387 - accuracy: 0.0000e+00 - 202ms/epoch - 40ms/step
Epoch 3/15
5/5 - 0s - loss: 1.5965 - accuracy: 0.2000 - 185ms/epoch - 37ms/step
Epoch 4/15
5/5 - 0s - loss: 1.4376 - accuracy: 0.6000 - 171ms/epoch - 34ms/step
Epoch 5/15
5/5 - 0s - loss: 1.2764 - accuracy: 0.8000 - 166ms/epoch - 33ms/step
Epoch 6/15
5/5 - 0s - loss: 1.1752 - accuracy: 1.0000 - 157ms/epoch - 31ms/step
Epoch 7/15
5/5 - 0s - loss: 1.0409 - accuracy: 1.0000 - 157ms/epoch - 31ms/step
Epoch 8/15
5/5 - 0s - loss: 0.9281 - accuracy: 1.0000 - 159ms/epoch - 32ms/step
Epoch 9/15
5/5 - 0s - loss: 0.8779 - accuracy: 0.8000 - 163ms/epoch - 33ms/step
Epoch 10/15
5/5 - 0s - loss: 0.8023 - accuracy: 1.0000 - 162ms/epoch - 32ms/step
Epoch 11/15
5/5 - 0s - loss: 0.7015 - accuracy: 1.0000 - 162ms/epoch - 32ms/step
Epoch 12/15
5/5 - 0s - loss: 0.6025 - accuracy: 1.0000 - 161ms/epoch - 32ms/step
Epoch 13/15
5/5 - 0s - loss: 0.530